In [6]:
# train.py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageOps

# --------------------------
# Preprocessing (same as app)
# --------------------------
def preprocess(pil_img):
    img = pil_img.convert("L")  # grayscale
    img = ImageOps.invert(img)  # white digit on black
    arr = np.array(img)

    # Binarize
    thresh = 30
    arr = (arr > thresh).astype(np.uint8) * 255

    ys, xs = np.where(arr > 0)
    if len(xs) == 0 or len(ys) == 0:
        return None
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1
    arr = arr[y0:y1, x0:x1]

    # Resize to 20x20 with aspect ratio
    h, w = arr.shape
    if h > w:
        new_h, new_w = 20, max(1, int(round(20 * w / h)))
    else:
        new_w, new_h = 20, max(1, int(round(20 * h / w)))
    digit = Image.fromarray(arr).resize((new_w, new_h), Image.BILINEAR)

    # Pad to 28x28
    canvas = Image.new("L", (28, 28))
    top = (28 - new_h) // 2
    left = (28 - new_w) // 2
    canvas.paste(digit, (left, top))

    out = np.array(canvas).astype("float32") / 255.0
    out = np.expand_dims(out, axis=-1)
    return out

# --------------------------
# Load MNIST & normalize
# --------------------------
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)
x_test  = np.expand_dims(x_test, -1)

# --------------------------
# Data Augmentation
# --------------------------
datagen = ImageDataGenerator(
    rotation_range=15,        # rotate up to ±15 degrees
    width_shift_range=0.15,   # horizontal shift up to 15%
    height_shift_range=0.15,  # vertical shift up to 15%
    shear_range=0.15,         # shear distortions
    zoom_range=0.2,           # zoom in/out
    fill_mode="nearest"       # fill empty pixels
)
datagen.fit(x_train)

# --------------------------
# Model
# --------------------------
model = keras.Sequential([
    layers.Conv2D(32, 3, activation="relu", input_shape=(28,28,1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),  # helps generalization
    layers.Dense(10, activation="softmax"),
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# --------------------------
# Train with augmentation
# --------------------------
batch_size = 128
epochs = 10

model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) // batch_size,
    validation_data=(x_test, y_test),
    epochs=epochs
)

# --------------------------
# Evaluate
# --------------------------
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")

# --------------------------
# Save model
# --------------------------
model.save("mnist_model.h5")
print("Model saved as mnist_model.h5")


Epoch 1/10


c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


468/468 ━━━━━━━━━━━━━━━━━━━━ 33s 67ms/step - accuracy: 0.6315 - loss: 1.0880 - val_accuracy: 0.9713 - val_loss: 0.0952
Epoch 2/10
  1/468 ━━━━━━━━━━━━━━━━━━━━ 16s 35ms/step - accuracy: 0.8750 - loss: 0.4068

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8750 - loss: 0.4068 - val_accuracy: 0.9720 - val_loss: 0.0921
Epoch 3/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.9095 - loss: 0.2954 - val_accuracy: 0.9827 - val_loss: 0.0566
Epoch 4/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9141 - loss: 0.2827 - val_accuracy: 0.9828 - val_loss: 0.0556
Epoch 5/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 43s 93ms/step - accuracy: 0.9338 - loss: 0.2184 - val_accuracy: 0.9842 - val_loss: 0.0500
Epoch 6/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9297 - loss: 0.1800 - val_accuracy: 0.9854 - val_loss: 0.0482
Epoch 7/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 46s 97ms/step - accuracy: 0.9464 - loss: 0.1753 - val_accuracy: 0.9829 - val_loss: 0.0515
Epoch 8/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9766 - loss: 0.0908 - val_accuracy: 0.9830 - val_loss: 0.0509
Epoch 9/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 41s 87ms/step - accuracy: 0.9529 - loss: 0.1526 - val_accuracy: 0.988

Test accuracy: 0.9894
Model saved as mnist_model.h5
